In [37]:
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
import random  
import pandas as pd
import numpy as np

In [38]:
def weighted_vote(prediction_set,oob_accuracy): #加權投票
    final_results = []
    for i in range(len(prediction_set[0])):
        predict_result = []
        for j in range(len(prediction_set)):
            predict_result.append( (prediction_set[j][i]) )
        score_0 ,score_1 ,score_2= 0 , 0 , 0
        for k in range(len(predict_result)):
            if predict_result[k] == 0:
                score_0 += oob_accuracy[k]
            elif predict_result[k] == 1:
                score_1 +=  oob_accuracy[k]
            else:
                score_2 += oob_accuracy[k]
        total_score = score_0 + score_1+ score_2 
        if score_0/total_score > 1/3:
            final_result = 0
        elif score_1/total_score > 1/3:
            final_result = 1
        else:
            final_result = 2
        final_results.append(final_result)
        
    return final_results

In [39]:
def Trees_Weighting_Random_Forest(training_x , training_y, test_x, max_depth, min_size, sample_size, n_trees, n_features):
    OOB_accuracy = []
    Prediction_set = []
    for i in range(n_trees):
        train_x, valid_x ,train_y ,valid_y = train_test_split(training_x, training_y , train_size = sample_size) #取後不放回的抽取部分資料建決策樹
        DT = DecisionTreeClassifier(max_depth = max_depth, min_samples_leaf = min_size,max_features=n_features)
        DT.fit(train_x,train_y)
        pred = DT.predict(valid_x)
        OOB_accuracy.append( accuracy_score( valid_y,pred))
        test_pred = DT.predict(test_x)
        Prediction_set.append(test_pred)
    
    weighted_result = weighted_vote(Prediction_set,OOB_accuracy)
    return weighted_result

In [59]:
wine = datasets.load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['label'] = wine.target
df['label'] = df['label'].astype(int)
X = df.drop(['label'],axis=1)
Y = df['label']

In [60]:
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,label
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [67]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
cnt = 1
rf_accuracy_set ,twrf_accuracy_set= [],[]

max_depth = 5
min_size = 1
sample_size = 0.8
n_trees = 300
n_features = 4

for train_index, valid_index in kf.split(X, Y):
    print('fold',cnt)
    train_x = X.iloc[train_index,:]
    train_y = df.iloc[train_index,:]['label']
    valid_x = X.iloc[valid_index,:]
    valid_y = df.iloc[valid_index,:]['label']

    TWRF_pred = Trees_Weighting_Random_Forest(train_x,train_y , valid_x , max_depth, min_size, sample_size, n_trees, n_features)
    twrf_accuracy_set.append(accuracy_score(valid_y,TWRF_pred))

    rf = RandomForestClassifier(max_depth = max_depth ,n_estimators= n_trees , max_features=n_features)
    rf.fit(train_x,train_y)
    rf_pred = rf.predict(valid_x)
    rf_accuracy_set.append(accuracy_score(valid_y,rf_pred))
    cnt += 1
    
print("Random forest 5-fold average accuracy" , np.mean(rf_accuracy_set) )
print("Trees Weighting Random Forest 5-fold average accuracy" , np.mean(twrf_accuracy_set) )

fold 1
fold 2
fold 3
fold 4
fold 5
Random forest 5-fold average accuracy 0.9828571428571429
Trees Weighting Random Forest 5-fold average accuracy 0.9942857142857143
